In [32]:
using JuMP
using Gurobi
using CSV
using DataFrames
using Statistics

# import Pkg; Pkg.add("HTTP")
using HTTP
# import Pkg; Pkg.add("JSON3")
using JSON3

In [33]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-08


In [4]:
classes_url = "https://fireroad.mit.edu/courses/all?full=true"

"https://fireroad.mit.edu/courses/all?full=true"

In [5]:
function get_all_classes()
    """
    Returns an array of dictionaries for each class that is valid, meaning it has a rating,
    in/out-of-class hours, and is public (ensuring that each class also has subject_id, title, and semester).
    """
    response = HTTP.get(classes_url)
    if response.status == 200
        all_classes_info_raw = JSON3.read(String(response.body))  # List of dictionaries
    else
        println("Failed to fetch all_classes_data: ", response.status)
        return []
    end

    # Identify "bad" classes missing required fields or not public
    bad_data = Set([
        class_info["subject_id"] for class_info in all_classes_info_raw
        if !haskey(class_info, "rating") ||
           !haskey(class_info, "in_class_hours") ||
           !haskey(class_info, "out_of_class_hours") ||
           class_info["public"] == false
    ])

    # Filter out bad classes
    all_classes_info = [
        class_info for class_info in all_classes_info_raw
        if !(class_info["subject_id"] in bad_data)
    ]

    return all_classes_info
end

get_all_classes (generic function with 1 method)

In [6]:
all_classes = get_all_classes()
println("Number of valid classes: ", length(all_classes))

Number of valid classes: 4703


In [12]:
all_classes[1]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 28 entries:
  :rating              => 5.4
  :gir_attribute       => "REST"
  :has_final           => false
  :description         => "Presents engineering problems in a computational set…
  :offered_fall        => false
  :offered_spring      => true
  :meets_with_subjects => ["1.001"]
  :instructors         => ["J. Williams"]
  :out_of_class_hours  => 7.99
  :total_units         => 12
  :related_subjects    => ["2.156", "1.205", "1.C51", "1.000", "1.631", "1.063"…
  :pdf_option          => false
  :in_class_hours      => 5.7
  :is_half_class       => false
  :level               => "U"
  :prerequisites       => "GIR:CAL1"
  :subject_id          => "1.00"
  :title               => "Engineering Computation and Data Science"
  :lab_units           => 2
  :design_units        => 0
  :public              => true
  :offered_summer      => false
  :lecture_units       => 3
  :

In [41]:
n = length(all_classes)
S = 8

8

In [27]:
R = [class[:rating] for class in all_classes];
H = [class[:in_class_hours] + class[:out_of_class_hours] for class in all_classes];
U = [class[:total_units] for class in all_classes];

In [25]:
mean(R)

6.049223899638529

## OPTIMIZER

In [42]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))

α = 0.8

# @variable(model, X[1:n,1:s] >=0)
@variable(model, X[1:n, 1:S], Bin)

@objective(model, Min, sum(α*H[i]*X[i,s] - (1-α)*R[i]*X[i,s] for i=1:n,s=1:S));

@constraint(model, [s in 1:S], sum(U[i]*X[i, s] for i=1:n) >= 36);
# @constraint(model, capacity_constraint[j in 1:m], sum(x[i,j] for i=1:n) <= capacity[j]);

optimize!(model)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 37624 columns and 33360 nonzeros
Model fingerprint: 0xf2a0b634
Variable types: 0 continuous, 37624 integer (37624 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [2e-02, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 4e+01]
Found heuristic solution: objective 127.6000000
Presolve removed 7 rows and 35219 columns
Presolve time: 0.12s
Presolved: 1 rows, 2405 columns, 2405 nonzeros
Variable types: 0 continuous, 2405 integer (1528 binary)
Found heuristic solution: objective 113.2000000

Root relaxation: objective 3.320000e+00, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

In [44]:
taken_classes = [i for i in 1:n if value.(X)[i] == 1.0]

8-element Vector{Int64}:
  144
  294
  296
  816
 1180
 2021
 2885
 3017

In [52]:
taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(X))]

8-element Vector{Vector{Int64}}:
 [144, 294, 296, 816, 1180, 2021, 2885, 3017]
 [144, 294, 296, 816, 1180, 2021, 2885, 3017]
 [144, 294, 296, 816, 1180, 2021, 2885, 3017]
 [144, 294, 816, 1180, 2021, 2885, 3017, 4347]
 [144, 294, 296, 816, 1180, 2021, 2885, 3017]
 [144, 294, 296, 816, 1180, 2021, 2885, 3017]
 [294, 714, 816, 1180, 2021, 2885, 3017, 4347]
 [144, 294, 296, 816, 1180, 2021, 2885, 3017]

In [50]:
value.(X)

4703×8 Matrix{Float64}:
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  ⋮                        ⋮         
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [51]:
value.(X)[:, 1]

4703-element Vector{Float64}:
 -0.0
 -0.0
  0.0
  0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
  0.0
 -0.0
 -0.0
  ⋮
 -0.0
  0.0
 -0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -0.0
  0.0
  0.0

## INTERPRETER

In [57]:
function interpret_course_nums(classes_id_per_sem)
    course_nums = [[all_classes[class_ind][:subject_id] for class_ind in sem] for sem in classes_id_per_sem]
    return course_nums
end

interpret_course_nums (generic function with 1 method)

In [58]:
interpret_course_nums(taken_classes_sem)

8-element Vector{Vector{String}}:
 ["1.713", "10.960", "10.991", "14.399", "15.839", "20.S900", "24.93", "3.903"]
 ["1.713", "10.960", "10.991", "14.399", "15.839", "20.S900", "24.93", "3.903"]
 ["1.713", "10.960", "10.991", "14.399", "15.839", "20.S900", "24.93", "3.903"]
 ["1.713", "10.960", "14.399", "15.839", "20.S900", "24.93", "3.903", "HST.533"]
 ["1.713", "10.960", "10.991", "14.399", "15.839", "20.S900", "24.93", "3.903"]
 ["1.713", "10.960", "10.991", "14.399", "15.839", "20.S900", "24.93", "3.903"]
 ["10.960", "12.834", "14.399", "15.839", "20.S900", "24.93", "3.903", "HST.533"]
 ["1.713", "10.960", "10.991", "14.399", "15.839", "20.S900", "24.93", "3.903"]